In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

df = pd.read_csv('data/jobs.csv')

### In the process of data tranformation, it was discovered that in some entries(rows) ids(id column) were skipped, so all values were shifted to the left.
### Filled in missing ids (since they go in order, same as indices) and shifted values to the right

In [2]:
# from this statement it was clear that values are shifted, since there are some date/time values
df['work_from_home'].value_counts()

True       5205
00:28.3       1
00:12.9       1
Name: work_from_home, dtype: int64

In [3]:
#another confirmation of the observation, we see that ids go in the same order as indices, however in the last 5 rows they are not the same
df.tail()

,id,title,company_name,location,via,extensions,posted_at,schedule_type,work_from_home,date_time,salary_pay,salary_rate,salary_avg,salary_min,salary_max,salary_hourly,salary_yearly,salary_standardized,description_tokens
11204,11201,Senior Data Analyst,Oracle,"Jefferson City, MO",via Trabajo.org,"['19 hours ago', 'Full-time', 'Health insuranc...",19 hours ago,Full-time,NaN,00:11.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['go', 'tableau', 't-sql', 'spreadsheet', 'sha..."
11205,11202,Principal Data Analyst,Fidelity Investments,"Tishomingo, OK",via JobiLike,"['18 hours ago', 'Full-time']",18 hours ago,Full-time,NaN,00:11.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['sap', 'snowflake', 'sql', 'hadoop', 'aws']"
11206,11203,Senior Data Analyst,Oracle,"Kansas City, MO",via Trabajo.org,"['13 hours ago', 'Full-time', 'Health insuranc...",13 hours ago,Full-time,NaN,00:11.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['go', 'sql']"
11207,11204,Principal Data Analyst,Fidelity Investments,"Marietta, OK",via JobiLike,"['18 hours ago', 'Full-time']",18 hours ago,Full-time,NaN,00:11.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['sap', 'snowflake', 'sql', 'hadoop', 'aws']"
11208,11205,HR Analytics PhD to Write Paper on ML Model fo...,Upwork,Anywhere,via Upwork,"['6 hours ago', '45 to 73 an hour', 'Work from...",6 hours ago,Contractor,True,00:11.6,45 to 73,an hour,59.0,45.0,73.0,59.0,NaN,122720.0,[]


In [4]:
#in order to find the missing ids, use pd.to_numeric() that attempts to convert the column values into number
#errors='coerce' parameter tells the function to replace non-numeric values with 'NaN'
#isnull() return true if numeric value is missing, so we get boolean Series as a result('id_non_numeric')
id_non_numeric = pd.to_numeric(df['id'],errors='coerce').isnull()
#in order to get a list of rows' indices, where shift is necessary, boolean Series serves as a filter for pulling the indices 
rows_to_shift = list(df[id_non_numeric]['id'].index)

In [5]:

rows_to_shift

[1794, 4687, 4688]

In [6]:
# we can see the values are shifted to the left, that's why id is missing
df.iloc[rows_to_shift[0]]

id                                                   Introduction to...
title                                                            Upwork
company_name                                                   Anywhere
location                                                     via Upwork
via                    ['17 hours ago', 'Work from home', 'Contractor']
extensions                                                 17 hours ago
posted_at                                                    Contractor
schedule_type                                                      True
work_from_home                                                  00:28.3
date_time                                                           NaN
salary_pay                                                          NaN
salary_rate                                                         NaN
salary_avg                                                          NaN
salary_min                                                      

In [7]:
# for each specific row shift values to the right and fill id with NaN
for r in rows_to_shift:
    df.iloc[r] = df.iloc[r].shift(periods=1,fill_value=np.nan)

In [8]:
#id is NaN, the rest of the values in correct spots
df.iloc[rows_to_shift[0]]

id                                                                  NaN
title                                                Introduction to...
company_name                                                     Upwork
location                                                       Anywhere
via                                                          via Upwork
extensions             ['17 hours ago', 'Work from home', 'Contractor']
posted_at                                                  17 hours ago
schedule_type                                                Contractor
work_from_home                                                     True
date_time                                                       00:28.3
salary_pay                                                          NaN
salary_rate                                                         NaN
salary_avg                                                          NaN
salary_min                                                      

In [9]:
#reset the dataframe index to a sequential integer index (new 'index' column appears)
#drop old 'id' column, rename new 'index' column into 'id'
df = df.reset_index().drop(columns=['id']).rename(columns={'index':'id'})

In [10]:
#id values are in sequential order
df

,id,title,company_name,location,via,extensions,posted_at,schedule_type,work_from_home,date_time,salary_pay,salary_rate,salary_avg,salary_min,salary_max,salary_hourly,salary_yearly,salary_standardized,description_tokens
0,0,Data Analyst Technical - Consultant,Intermountain Healthcare,"Oklahoma City, OK",via Monster,"['19 hours ago', 'Full-time', 'Health insurance']",19 hours ago,Full-time,NaN,00:10.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['vba', 'spreadsheet', 'javascript', 'word', '..."
1,1,"Sr Analyst, Data Engineer",Estee Lauder Companies,Oklahoma,via ZipRecruiter,"['12 hours ago', 'Full-time']",12 hours ago,Full-time,NaN,00:10.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
2,2,Legal Data Analyst,DISYS,United States,via Indeed,"['4 hours ago', '21 to 23 an hour', 'Contractor']",4 hours ago,Contractor,NaN,00:10.1,21 to 23,an hour,22.0,21.0,23.0,22.0,NaN,45760.0,[]
3,3,Data analyst,Modis,United States,via BeBee,"['13 hours ago', 'No degree mentioned']",13 hours ago,NaN,NaN,00:10.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
4,4,Data Analyst (Risk Adjustment Consulting Resea...,Cambia Health Solutions,United States,via LinkedIn,"['23 hours ago', 'Full-time', 'Health insuranc...",23 hours ago,Full-time,NaN,00:10.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['sas', 'matlab', 'sql', 'spss', 'r']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11204,11204,Senior Data Analyst,Oracle,"Jefferson City, MO",via Trabajo.org,"['19 hours ago', 'Full-time', 'Health insuranc...",19 hours ago,Full-time,NaN,00:11.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['go', 'tableau', 't-sql', 'spreadsheet', 'sha..."
11205,11205,Principal Data Analyst,Fidelity Investments,"Tishomingo, OK",via JobiLike,"['18 hours ago', 'Full-time']",18 hours ago,Full-time,NaN,00:11.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['sap', 'snowflake', 'sql', 'hadoop', 'aws']"
11206,11206,Senior Data Analyst,Oracle,"Kansas City, MO",via Trabajo.org,"['13 hours ago', 'Full-time', 'Health insuranc...",13 hours ago,Full-time,NaN,00:11.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['go', 'sql']"
11207,11207,Principal Data Analyst,Fidelity Investments,"Marietta, OK",via JobiLike,"['18 hours ago', 'Full-time']",18 hours ago,Full-time,NaN,00:11.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['sap', 'snowflake', 'sql', 'hadoop', 'aws']"


### In the next step we need to convert 'description_tokens'(contains lists of various skils) into multiple columns. Each new column will only have one skill as an attribute.If the skiil is required it will be represented as 1, no required - 0

In [11]:
# we see that 'description_tokens' have some Nulls, we need to skip the nulls in the transformation
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11209 entries, 0 to 11208
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   11209 non-null  int64  
 1   title                11209 non-null  object 
 2   company_name         11206 non-null  object 
 3   location             11195 non-null  object 
 4   via                  11206 non-null  object 
 5   extensions           11206 non-null  object 
 6   posted_at            11206 non-null  object 
 7   schedule_type        11166 non-null  object 
 8   work_from_home       5207 non-null   object 
 9   date_time            11206 non-null  object 
 10  salary_pay           2229 non-null   object 
 11  salary_rate          2229 non-null   object 
 12  salary_avg           2229 non-null   object 
 13  salary_min           2097 non-null   float64
 14  salary_max           2097 non-null   float64
 15  salary_hourly        1183 non-null  

In [12]:
#create a boolean filter to indicate which rows have no null in the 'description_tokens'
mask = df['description_tokens'].notnull()
#create an instance of MultiLabelBinarizer object
mlb = MultiLabelBinarizer()

#extract values from df['description_tokens'] where value is true, dropping any null values from the column
#remove square brackets from the strings and split into separate strings using ',' as delimiter
# fit_transform creates a binary matrix on filtered subset of the data frame
# new dataframe is created from the binary matrix
#reindex() allows to make sure that each skill in the new dataframe has a corresponding inex from the original df
skills_dummies = pd.DataFrame(mlb.fit_transform(df.loc[mask,'description_tokens'].dropna().str.strip('[]').str.split(',')),columns=mlb.classes_).reindex(df.index,fill_value=0)


In [13]:
#because of some entries that had an empty list in the description_tokens column, we ended up with an empty column(it was considered as a separate value)
mlb.classes_

array(['', " 'airflow'", " 'alteryx'", " 'apl'", " 'asp.net'",
       " 'assembly'", " 'atlassian'", " 'aurora'", " 'aws'", " 'azure'",
       " 'bash'", " 'bigquery'", " 'bitbucket'", " 'c'", " 'c++'",
       " 'c/c++'", " 'cognos'", " 'css'", " 'dax'", " 'docker'",
       " 'dplyr'", " 'excel'", " 'fortran'", " 'gcp'", " 'gdpr'",
       " 'ggplot2'", " 'git'", " 'github'", " 'gitlab'", " 'go'",
       " 'golang'", " 'graphql'", " 'hadoop'", " 'html'", " 'java'",
       " 'javascript'", " 'jira'", " 'jquery'", " 'js'", " 'julia'",
       " 'jupyter'", " 'keras'", " 'linux'", " 'linux/unix'", " 'looker'",
       " 'matlab'", " 'matplotlib'", " 'microstrategy'", " 'mongo'",
       " 'mongodb'", " 'mssql'", " 'mysql'", " 'no-sql'", " 'node'",
       " 'node.js'", " 'nosql'", " 'numpy'", " 'outlook'", " 'pandas'",
       " 'perl'", " 'php'", " 'pl/sql'", " 'plotly'", " 'postgres'",
       " 'postgresql'", " 'power_bi'", " 'powerpoint'", " 'powerpoints'",
       " 'powershell'", " 'pyspark

In [14]:
#Therefore, remove this column
skills_dummies.drop(columns=[''],inplace=True)

In [15]:
#as it was instructed adding 'desription_token_' string to each skill
skills_dummies.columns = ['description_token_' + col.replace("'","") for col in skills_dummies.columns]


In [16]:
skills_dummies

,description_token_ airflow,description_token_ alteryx,description_token_ apl,description_token_ asp.net,description_token_ assembly,description_token_ atlassian,description_token_ aurora,description_token_ aws,description_token_ azure,description_token_ bash,...,description_token_tableau,description_token_tensorflow,description_token_terminal,description_token_unix,description_token_vb.net,description_token_vba,description_token_visio,description_token_visual_basic,description_token_vue.js,description_token_word
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11204,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
11205,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11206,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11207,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
#combine original dataframe with newly created skills dataframe
# the indices of original df wre preserved for skills_dummies df, so concatination is available
df_new = pd.concat([df, skills_dummies], axis=1)

In [18]:
df_new

,id,title,company_name,location,via,extensions,posted_at,schedule_type,work_from_home,date_time,...,description_token_tableau,description_token_tensorflow,description_token_terminal,description_token_unix,description_token_vb.net,description_token_vba,description_token_visio,description_token_visual_basic,description_token_vue.js,description_token_word
0,0,Data Analyst Technical - Consultant,Intermountain Healthcare,"Oklahoma City, OK",via Monster,"['19 hours ago', 'Full-time', 'Health insurance']",19 hours ago,Full-time,NaN,00:10.1,...,0,0,0,0,0,1,0,0,0,0
1,1,"Sr Analyst, Data Engineer",Estee Lauder Companies,Oklahoma,via ZipRecruiter,"['12 hours ago', 'Full-time']",12 hours ago,Full-time,NaN,00:10.1,...,0,0,0,0,0,0,0,0,0,0
2,2,Legal Data Analyst,DISYS,United States,via Indeed,"['4 hours ago', '21 to 23 an hour', 'Contractor']",4 hours ago,Contractor,NaN,00:10.1,...,0,0,0,0,0,0,0,0,0,0
3,3,Data analyst,Modis,United States,via BeBee,"['13 hours ago', 'No degree mentioned']",13 hours ago,NaN,NaN,00:10.1,...,0,0,0,0,0,0,0,0,0,0
4,4,Data Analyst (Risk Adjustment Consulting Resea...,Cambia Health Solutions,United States,via LinkedIn,"['23 hours ago', 'Full-time', 'Health insuranc...",23 hours ago,Full-time,NaN,00:10.1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11204,11204,Senior Data Analyst,Oracle,"Jefferson City, MO",via Trabajo.org,"['19 hours ago', 'Full-time', 'Health insuranc...",19 hours ago,Full-time,NaN,00:11.6,...,0,0,0,0,0,0,0,0,0,0
11205,11205,Principal Data Analyst,Fidelity Investments,"Tishomingo, OK",via JobiLike,"['18 hours ago', 'Full-time']",18 hours ago,Full-time,NaN,00:11.6,...,0,0,0,0,0,0,0,0,0,0
11206,11206,Senior Data Analyst,Oracle,"Kansas City, MO",via Trabajo.org,"['13 hours ago', 'Full-time', 'Health insuranc...",13 hours ago,Full-time,NaN,00:11.6,...,0,0,0,0,0,0,0,0,0,0
11207,11207,Principal Data Analyst,Fidelity Investments,"Marietta, OK",via JobiLike,"['18 hours ago', 'Full-time']",18 hours ago,Full-time,NaN,00:11.6,...,0,0,0,0,0,0,0,0,0,0


### In the 'work_from_home' column there are some True values and the rest are unknown. It's wrong to assume that these values are False, because we don't know exactly.This assumption may create bias. However, for the sake of further processing of this data (as it was instructed) we fill in the missing values with False.

In [19]:

df_new['work_from_home'].fillna(value=False, inplace=True)

In [20]:
#csv file to be used in eda.ipynb
df_new.to_csv('data/skills_features.csv', index=False)

In [21]:
#check how many null values in the salary_standardized column before dropping them
df["salary_standardized"].isna().sum()

8980

In [22]:
#drop the rows where'salary_standardized' is Null
df_new.dropna(subset = ['salary_standardized'], inplace = True)

In [23]:
df_new

,id,title,company_name,location,via,extensions,posted_at,schedule_type,work_from_home,date_time,...,description_token_tableau,description_token_tensorflow,description_token_terminal,description_token_unix,description_token_vb.net,description_token_vba,description_token_visio,description_token_visual_basic,description_token_vue.js,description_token_word
2,2,Legal Data Analyst,DISYS,United States,via Indeed,"['4 hours ago', '21 to 23 an hour', 'Contractor']",4 hours ago,Contractor,False,00:10.1,...,0,0,0,0,0,0,0,0,0,0
11,11,Lead Data Analyst,Laka & Company,"Bentonville, AR",via LinkedIn,"['10 hours ago', '100K to 140K a year', 'Full-...",10 hours ago,Full-time,False,00:12.6,...,0,0,0,0,0,0,0,0,0,0
20,20,Junior Data Analyst,Educated Solutions Corp,United States,via My Stateline Jobs,"['8 hours ago', '25 to 26 an hour', 'Full-time...",8 hours ago,Full-time,False,00:14.4,...,0,0,0,0,0,0,0,0,0,0
58,58,Healthcare Data Analyst,Pediatrics Northwest,United States,via Job,"['4 hours ago', '25 an hour', 'Full-time', 'He...",4 hours ago,Full-time,False,00:21.9,...,0,0,0,0,0,0,0,0,0,0
63,63,Program Management Data Analyst,Citi,United States,via Big Country Jobs,"['8 hours ago', '73,570 to 110,350 a year', 'F...",8 hours ago,Full-time,False,00:24.4,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11177,11177,SPSS ANCOVA analysis & psychological write up,Upwork,Anywhere,via Upwork,"['23 hours ago', '25 to 35 an hour', 'Work fro...",23 hours ago,Contractor,True,00:42.4,...,0,0,0,0,0,0,0,0,0,0
11182,11182,Fix Google Indexing Issue and set up analytics,Upwork,Anywhere,via Upwork,"['11 hours ago', '16 to 45 an hour', 'Work fro...",11 hours ago,Contractor,True,00:42.4,...,0,0,0,0,0,0,0,0,0,0
11197,11197,Statistician spss,Upwork,Anywhere,via Upwork,"['14 hours ago', '10 to 15 an hour', 'Work fro...",14 hours ago,Contractor,True,00:09.7,...,0,0,0,0,0,0,0,0,0,0
11203,11203,Sales analysis on raw data for an E-Commerce S...,Upwork,Anywhere,via Upwork,"['17 hours ago', '16 to 45 an hour', 'Work fro...",17 hours ago,Contractor,True,00:09.7,...,0,0,0,0,0,0,0,0,0,0


In [24]:
#csv file to be used in predict.ipynb
df_new.to_csv('data/clean_predict.csv',index=False)